# Batch Process multiple freemocap session folders


In [ ]:
#get paths to folder full of session folders
list_of_session_folders = list(path_to_folder_of_session_folders.glob('ses*'))
print(list_of_session_folders)

In [ ]:
#import stuff
from pathlib import Path


from src.core_processes.batch_process_session_folders import process_session_folder
from src.core_processes.capture_volume_calibration.anipose_camera_calibration import freemocap_anipose


## input `Path to folder of session folders` HERE


In [ ]:

path_to_folder_of_session_folders = Path(
    r'D:\Dropbox\Northeastern\Courses\Biol2299\2022_09_Fall_Biol2299_Inquiries_NeuralControlOfRealWorldBehavior\freemocap_bos_com_standing_data\FreeMocap_Data')


## Input `path_to_camrera_calibration_toml` HERE

In [ ]:

path_to_camera_calibration_toml = Path(
    r"D:\Dropbox\Northeastern\Courses\Biol2299\2022_09_Fall_Biol2299_Inquiries_NeuralControlOfRealWorldBehavior\freemocap_bos_com_standing_data\FreeMocap_Data\sesh_2022-09-28_15_57_08_calibration\sesh_2022-09-28_15_57_08_calibration_calibration.toml"
)


## Input `path_to_blender_executable` HERE

In [ ]:
path_to_blender_executable = Path(r"C:\Program Files\Blender Foundation\Blender 3.2\blender.exe")

In [ ]:
# load anipose calibration object
anipose_calibration_object = freemocap_anipose.CameraGroup.load(str(path_to_camera_calibration_toml))

In [ ]:
for session_folder_path in list_of_session_folders:
    print(f'-------------------------Processing {session_folder_path}-------------------------')
    if Path(session_folder_path / "synchronized_videos").exists():  # freemocap version > v0.0.54 (aka `alpha`)
        synchronized_videos_folder = Path(session_folder_path) / 'synchronized_videos'
    if Path(session_folder_path / "SyncedVideos").exists():  # freemocap version <= v0.0.54 (aka `pre-alpha`)
        synchronized_videos_folder = Path(session_folder_path) / 'SyncedVideos'
    else:
        print(f'No folder full of synchronized videos found for {session_folder_path}')
        continue

    output_data_folder = Path(session_folder_path) / 'output_data'
    output_data_folder.mkdir(exist_ok=True, parents=True)

    # process session folder via 'standard' freemocap pipeline
    process_session_folder(synchronized_videos_folder=synchronized_videos_folder,
                           output_data_folder=output_data_folder,)

print("Done!")